# Chain

- 여러 컴포넌트들을 논리적 순서대로 연결하여 복잡한 작업을 수행하는 구조로 복잡한 AI 작업을 체계적이고 효율적으로 구현할 수 있게 해준다.
- 기본 개념
    - 일련의 작업을 구성하는 여러 개별 컴포넌트들을 정의된 순서대로 실행시킨다.
    - 단일 API 호출을 넘어 여러 호출을 논리적 순서로 연결 가능하다.
    - 복잡한 작업을 작은 단계로 분해하여 순차적으로 처리할 수 있다.

- Langchain은 `off-the-shelf chains` 방식과 `LCEL(Langchain Expression Language)`  두가지 방식이 있다.
  - off-the-shelf chains 방식
    - 미리정의된 Chain 클래스를 사용해 체인을 구성하는 방식
    - Langchain의 초기 방식으로 대부분의 class들이 deprecated 되었다.
  - LECL 방식
    - 표현식을 이용해 체인을 구성하는 방식이다.
    - 현재 LangChain은 LCEL(LangChain Expression Language)을 중심으로 발전하고 있다


# Off-the-shelf chains 예제

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from pprint import pprint

prompt_template = PromptTemplate(
    template="{item}에 어울리는 이름 {count}개를 만들어 주세요."
)
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=1.0
)

# 순서
## 1. prompt생성 -> 2. llm 요청
prompt = prompt_template.invoke({"item":"스마트폰", "count": 5})
# prompt.text
result = model.invoke(prompt)
print(result)

content='물론이죠! 스마트폰에 어울리는 이름 5개를 제안해드릴게요.\n\n1. **소닉폰 (Sonic Phone)** - 빠르고 효율적인 성능을 강조하는 이름.\n2. **넥서스 (Nexus)** - 연결과 통합을 의미하는 이름으로, 다양한 기능을 포괄하는 스마트폰에 적합.\n3. **스텔라 (Stella)** - 별을 의미하여, 반짝이는 디자인과 성능을 나타내는 이름.\n4. **비전포 (VisionPro)** - 뛰어난 디스플레이와 카메라 기능을 강조하는 이름.\n5. **에코스 (Echos)** - 사용자와의 소통과 반향을 중시하는 스마트폰에 잘 어울리는 이름.\n\n필요한 정보가 더 있으시면 말씀해 주세요!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 183, 'prompt_tokens': 22, 'total_tokens': 205, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None} id='run-6bfca022-8a3a-41ab-8a50-80b14b9d3cfe-0' usage_metadata={'input_tokens': 22, 'output_tokens': 183, 'total_tokens': 205, 'input_token_details': {'audio': 0, 'cache_read': 0},

In [4]:
from langchain import LLMChain

chain = LLMChain(
    prompt=prompt_template,
    llm=model
    # , output_parser=OutputParser객체
)
# 입력: prompt_template의 전달할 값 -> chain(prompt_template -> llm) -> 출력:llm의 결과
result = chain.invoke({"item":"가방", "count":3})

C:\Users\Playdata\AppData\Local\Temp\ipykernel_4532\22492021.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [5]:
print(result['text'])

물론이죠! 가방에 어울리는 이름 3개를 제안해 드릴게요.

1. **모던 스퀘어** - 세련된 디자인의 가방에 잘 어울리는 이름으로, 도시적인 느낌을 줍니다.
2. **에코 투고** - 자연친화적인 소재로 만든 가방에 적합한 이름으로, 환경을 생각하는 소비자에게 어필할 수 있습니다.
3. **루나 클러치** - 우아하고 스타일리시한 클러치 백에 잘 어울리는 이름으로, 밤 외출 시 사용하기 좋습니다.

이 이름들이 도움이 되길 바랍니다!


In [6]:
# LCEL
chain2 = prompt_template | model 
result2 = chain2.invoke({"item":"컴퓨터", "count": 5})

In [7]:
print(result2.content)

물론입니다! 여기 컴퓨터에 어울리는 이름 5개를 제안드립니다:

1. **코드마스터 (CodeMaster)**
2. **데이터스미스 (DataSmith)**
3. **알고리즘스 (Algorithmus)**
4. **바이트비전 (ByteVision)**
5. **네트워크우즈 (NetworkWiz)**

이 이름들이 마음에 드시길 바랍니다!


# LCEL (LangChain Expression Language)
- LCEL은 LangChain의 핵심 기능인 **체인(Chain)을 더욱 효율적으로 구현하기 위해 도입된 **선언적 방식의 체인(chain) 구성 언어**이다.
- `|` 연산자를 이용해 선언적 방법으로 Chain을 만든다.
- [Runnable](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html) type의 component들이 chain에 포함될 수있다.
    - `|` 연산자를 이용해 Runnable들을 연결한다.
    - chain이 실행되면 각 Runnable의 invoke() 메소드가 실행된다. 그리고 invoke()의 리턴값을 다음 Runnable의 invoke()에 전달해서 실행시킨다.
    - [Runnable 컴포넌트별 입출력 타입](https://python.langchain.com/docs/expression_language/interface)
        - 각 컴포넌트의 input과 output 타입에 맞춰 값이 전달되도록 한다.
- https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel

## Runnable
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 체인(chain) 형태로 연결하여 복잡한 작업을 수행할 수 있게 한다.
- Chain을 구성하는 class들은 Runnable의 하위 클래스로 구현한다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 요청)을 수행하는 독립적인 컴포넌트이다.
    - LangChain의 다양한 컴포넌트(PromptTemplate, LLM, OutputParser 등)들이 Runnable을# Runnable
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 체인(chain) 형태로 연결하여 복잡한 작업을 수행할 수 있게 한다.
- Chain을 구성하는 class들은 Runnable의 하위 클래스로 구현한다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 요청)을 수행하는 독립적인 컴포넌트이다.
    - LangChain의 다양한 컴포넌트(PromptTemplate, LLM, OutputParser 등)들이 Runnable을 상속받아 구현된다.
- 체인 연결 및 작업 흐름 관리:
    - Runnable은 파이프라인처럼 체인(순차적으로 실행되는 작업들을 연결한 것)을 구성할 수 있으며, `|` 연산자를 사용해 간단히 연결 가능하다.
    - 입력과 출력 형식을 통일해서 컴포넌트를 매끄럽게 연결한다
- 모듈화 및 디버깅 용이성:
    - 각 단계가 명확히 분리되어 디버깅 및 유지보수가 용이하다.
    - 복잡한 작업을 작은 단위로 나누어 관리할 수 있다.
### Runnable의 표준 메소드
- 모든 Runnable이 구현하는 공통 메소드
- `invoke()`: 입력 데이터를 처리하여 결과를 반환.
- `batch()`: 여러 입력 데이터들을 한 번에 처리.
- `stream()`: 스트리밍 방식으로 응답 반환.
- `ainvoke()`: 비동기 호출 지원.

### Runnable의 주요 구현체
- **`RunnablePassThrough`**
    - 입력데이터를 다음 chain으로 그대로 전달하거나, 필요에 따라 추가적인 key-value 쌍을 더해서 전달한다. 
- **`RunnableParallel`**
    - 여러 Runnable을 병렬로 실행하고 결과들을 합쳐서 다음 chain으로 전달한다.`**
- **`RunnableLambda`**
    - 일반 함수나 lambda 함수를 Runnable로 만들 때 사용.

In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [37]:
### 사용자 정의 Runnable class
from langchain_core.runnables import Runnable

class MyRunnable(Runnable):
    # invoke(): 1개 파라미터는 필수. -> Runnable이 작업할 때 필요한 입력 값.
    def invoke(self, input_data:str, config=None):
        # 이 Runnable이 해야하는 일을 invoke()에 구현.
        ## config: 추가 설정 정보들. RunnableConfig 타입 받는다.-> 호출할 때 dict로 전달.
        ### config에 어떤 값을 어떤 key로 넣을 지는 구현하는 쪽에서 결정.
        ### lang:언어코드 => 그 코드에 맞는 prompt 문장을 반환하도록 구현.
        if config is not None:
            # config : {"configurable": {"설정key":"설정값"}} -> chain에서 호출시 config 전달형식
            if config['configurable']['lang'] == "en":
                return f"Explain {input_data} in one sentence."
        return f"{input_data}에 대해서 한문장으로 설명해줘."
    
my_runnable = MyRunnable()
my_runnable.invoke("사과")
my_runnable.invoke("아이폰", {"configurable":{"lang":"en"}})

'Explain 아이폰 in one sentence.'

In [38]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", max_tokens=100)
prompt = my_runnable.invoke("사과")
res = model.invoke(prompt)
print(res.content)

사과는 달콤하고 상큼한 맛을 가진 과일로, 건강에 좋은 영양소와 항산화 물질이 풍부하여 다양한 요리에 활용됩니다.


In [39]:
from langchain_core.output_parsers import StrOutputParser
chain = my_runnable | model | StrOutputParser()
result = chain.invoke("Galaxy s24", {"lang":"en"})
# chain에서 Runnable로 config를 전달할 때: config={"configurable":{"lang":"en"}})

In [40]:
print(result)

The Samsung Galaxy S24 is an upcoming flagship smartphone expected to feature advanced technology, improved camera capabilities, and enhanced performance, building on the success of its predecessors in the Galaxy S series.


In [34]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# 실행 순서: prompt -> model -> outputParser
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)
parser = CommaSeparatedListOutputParser()
# system: 가이드, human: 질문
prompt_template = ChatPromptTemplate(
    messages=[
        ("system", "{format_instruction}.\n목록의 item은 {count}개를 넘지 안도록 해주세요."),
        ("human", "{query}")
    ],
    partial_variables={"format_instruction":parser.get_format_instructions()}
)

chain = prompt_template | model | parser

result = chain.invoke({"count":5, "query":"서울에 가 볼만한 여행지를 알려줘."})

In [35]:
print(result)

['경복궁', '남산타워', '인사동', '홍대', '동대문 디자인 플라자']


In [59]:
# 레시피 요청 -(llm)-> 영어 레시피  한국어로 번역 요청 -llm-> 한국어 레시피
# 1. chain(레시피를 알려주는 chain): 레시피요청 -> 레시피 출력(영어)
# 2. chain(번역하는 chain): 영어 -> 한국어 번역
# 3. 최종 chain: 레시피체인 -> 번역체인

##### 레시피 체인
model = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0.1
)
chef_template = ChatPromptTemplate(
    [
        ("system", "You are world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
        ("human", "I want to cook {food} food.")
    ]
)

chef_chain = chef_template | model | StrOutputParser()

In [44]:
res = chef_chain.invoke({"food":"steak"})

In [ ]:
print(res.content)

In [52]:
##### 번역 chain
translate_template = ChatPromptTemplate(
    [
        ("system", "당신은 번역가 입니다. 다음 내용을 한국어로 번역해 주세요."),
        ("human", "{query}")
    ]
)

translate_chain = translate_template | model | StrOutputParser()

# res = translate_chain.invoke({"query":"You are world-class international chef."})
res = translate_chain.invoke({"query":"Ich bin hungrig."})

In [53]:
print(res)

나는 배고파.


In [60]:
# chain도 Runnble Type -> 다른 chain의 컴포넌트로 포함될 수있다.
final_chain = chef_chain | translate_chain

In [61]:
result = final_chain.invoke({"food":"steak"})

In [62]:
print(type(result))
print(result)

<class 'str'>
좋은 선택입니다! 스테이크는 맛있고 다양한 방식으로 조리할 수 있는 요리입니다. 여기 클래식한 팬 시어드 스테이크와 마늘 버터를 위한 간단하고 쉬운 레시피가 있습니다. 이 레시피는 2인분입니다.

### 마늘 버터를 곁들인 팬 시어드 스테이크

#### 재료:
- 리브아이 또는 등심 스테이크 2개 (약 1인치 두께)
- 소금과 갓 간 블랙 페퍼
- 올리브 오일 2큰술
- 무염 버터 3큰술
- 다진 마늘 3쪽
- 장식용 신선한 허브 (타임이나 로즈마리 등, 선택 사항)

#### 조리 방법:

1. **스테이크 준비하기:**
   - 스테이크를 냉장고에서 꺼내어 실온에서 약 30분간 두세요. 이렇게 하면 더 고르게 익습니다.
   - 종이 타올로 스테이크의 수분을 제거하고, 양면에 소금과 갓 간 블랙 페퍼를 넉넉히 뿌려주세요.

2. **팬 가열하기:**
   - 중-강 불로 큰 팬(가급적 주철 팬)에 올리브 오일을 넣고 가열합니다. 오일이 반짝이지만 연기가 나지 않을 정도로 가열하세요.

3. **스테이크 시어링하기:**
   - 뜨거운 팬에 조심스럽게 스테이크를 놓습니다. 팬이 너무 붐비지 않도록 하세요; 필요하다면 한 번에 하나씩 조리하세요.
   - 스테이크를 한쪽에서 약 4-5분간 움직이지 않고 시어링합니다. 이렇게 하면 맛있는 크러스트가 생깁니다.

4. **뒤집고 버터 추가하기:**
   - 집게를 사용하여 스테이크를 뒤집습니다. 팬에 버터와 다진 마늘을 추가합니다.
   - 버터가 녹으면서 팬을 약간 기울이고 숟가락을 사용하여 녹은 버터를 스테이크 위에 뿌려줍니다. 이렇게 하면 풍미가 더해지고 스테이크의 윗부분이 익는 데 도움이 됩니다.

5. **원하는 익힘 정도로 조리하기:**
   - 원하는 익힘 정도에 따라 스테이크를 추가로 3-5분간 더 조리합니다:
     - 레어: 125°F (51°C)
     - 미디엄 레어: 135°F (57°C)
     - 미디엄: 145°F (63°C)
     - 미디엄 웰: 15

## 사용자 함수를 chain으로 정의
- 임의의 함수를 Runnable로 정의 할 수있다.
    - LangChain 에서 제공하지 않는 기능을 Chain으로 만들 때 유용한다.
- LangChain에서는 Runnable로 사용되는 사용자 정의 함수를 **Runnable Lambda** 라고 한다.
- 함수를 Runnable 로 명시하는데는 다음 두가지 방법이 있다.
1. `RunnableLambda` 이용
   - `RunnableLambda(함수)`
3. `@chain` 데코레이터 이용
   - ```python
     @chain
     def func():
         ...
    ```
### Runnable 로 정의 하는 함수 정의
- 이전 Chain의 출력을 입력 받는 파라미터를 한개 선언한다.
- 만약 함수가 여러개의 인자를 받는 경우 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼 함수를 작성하여 Runnable로 만든다.
```python
def plus(num1, num2):
    ...

def wrapper_plus(nums:dict|list):
    return plus(nums['num1'], nums['num2'])
```
- Chain의 실행결과를 return 한다.

## Chain 간의 연결

# Cache

- 응답 결과를 저장해서 같은 질문이 들어오면 LLM에 요청하지 않고 저장된 결과를 보여주도록 한다.
    - 처리속도와 비용을 절감할 수 있다.
    - 특히 chatbot같이 비슷한 질문을 하는 경우 유용하다.
- 저장 방식은 `메모리`, `sqlite` 등 다양한 방식을 지원한다.
    - https://python.langchain.com/docs/integrations/llms/llm_caching
```python
set_llm_cache(Cache객체)
```